In [32]:
import random

import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, log_loss, roc_auc_score

from src.dataset import get_dataset
from src.models.dualemb import DualEmbPredictor
from src.models.elog import ELOgPredictor
from src.models.freq import FrequencyMatchPredictor
from src.models.uni import UniformMatchPredictor

In [2]:
ITERATIONS = 1
random.seed(5438)
np.random.seed(5438)

In [10]:
def determine_target(row):
    if row["team_score"] > row["opponent_score"]:
        return 0
    elif row["team_score"] == row["opponent_score"]:
        return 1
    else:
        return 2

In [11]:
dataset = get_dataset()

/mnt/c/Personal/football-matches-prediction/src/dataset.py:47: DtypeWarning: Columns (11,14,50,102,123,125,129,131,133,135,137,139,141,143,145,147,149,157,158,160,161,163,165,167,169,171,173,184,186,187,205,206) have mixed types. Specify dtype option on import or set low_memory=False.
  mls_df = pd.read_csv("data/mls_matches.csv")


In [12]:
dataset

,team_id,opponent_id,team_at_home,opponent_at_home,team_score,opponent_score,fold
0,Scotland,England,1.0,0.0,0,0,international
1,England,Scotland,1.0,0.0,4,2,international
2,Scotland,England,1.0,0.0,2,1,international
3,England,Scotland,1.0,0.0,2,2,international
4,Scotland,England,1.0,0.0,3,0,international
...,...,...,...,...,...,...,...
143000,Sport Lisboa e Benfica,Sporting Clube de Braga,1.0,0.0,3,0,europe
143001,Panathinaikos Athlitikos Omilos,APS Atromitos Athinon,1.0,0.0,2,1,europe
143002,Fulham Football Club,Watford FC,1.0,0.0,4,1,europe
143003,Panthessalonikios Athlitikos Omilos Konstantin...,Athlitiki Enosi Konstantinoupoleos,1.0,0.0,1,1,europe


In [25]:
model_classes = [
    FrequencyMatchPredictor,
    UniformMatchPredictor,
    ELOgPredictor,
    # DualEmbPredictor,
]
# folds_names = ["brazil", "libertadores", "mls", "europe", "international"]
folds_names = ["brazil", "international"]

In [26]:
folds_train = [dataset[dataset["fold"] != name] for name in folds_names]
folds_test = [dataset[dataset["fold"] == name] for name in folds_names]

In [27]:
results = pd.DataFrame({}, columns=["metric", "model", "fold", "iteration", "value"])

In [ ]:
for iteration in range(ITERATIONS):
    for model_class in model_classes:
        for fold_train, fold_test, fold_test_name in zip(
            folds_train, folds_test, folds_names
        ):
            X_train = fold_train[["team_id", "opponent_id", "team_at_home", "opponent_at_home"]]
            y_train = fold_train[["team_score", "opponent_score"]]
            X_test = fold_test[["team_id", "opponent_id", "team_at_home", "opponent_at_home"]]
            y_test = fold_test[["team_score", "opponent_score"]]
            model = model_class()
            model.fit(X_train, y_train)
            pred = model.predict_and_update(X_test, y_test)
            max_pred = np.argmax(pred, axis=1)
            target = fold_test.apply(determine_target, axis=1).to_numpy()
            report = classification_report(
                target, max_pred, target_names=["win", "draw", "loss"], output_dict=True
            )
            metrics = {
                "accuracy": report["accuracy"],
                "log_loss": log_loss(target, pred, labels=[0, 1, 2]),
                "auc_roc": roc_auc_score(target, pred, average="micro", multi_class="ovr"),
                "weighted_precision": report["weighted avg"]["precision"],
                "weighted_recall": report["weighted avg"]["recall"],
            }
            for key, value in metrics.items():
                results.loc[len(results)] = {
                    "metric": key,
                    "model": model_class.__name__,
                    "fold": fold_test_name,
                    "iteration": iteration + 1,
                    "value": value,
                }

/mnt/c/Personal/football-matches-prediction/src/models/freq.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["target"] = df_y.apply(
/home/razevedo/.cache/pypoetry/virtualenvs/football-matches-prediction-ljJPvyK7-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/razevedo/.cache/pypoetry/virtualenvs/football-matches-prediction-ljJPvyK7-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 

In [ ]:
results

In [ ]:
results.groupby(["metric", "model", "fold"])["value"].mean().reset_index().groupby(
    ["metric", "model"]
)["value"].mean().reset_index().pivot(index="model", columns="metric", values="value")